In [22]:
import pymongo
import math 
# import re
import pandas as pd
# import numpy as np
import pylab 
from matplotlib import pyplot as plt
# from scipy import stats 
from  qgrid import show_grid as show 
from datetime import datetime
from powerapi.formula import RAPLFormulaHWPCReportHandler, FormulaState
from powerapi.report import HWPCReport


# import seaborn as sns

class testCase(object): 
    """in general its just the name of the machine where we launched the test """
    def __init__ (self , testname,database='rapls2',serveraddr='172.16.45.8',serverport=27017): 
        self._client = pymongo.MongoClient(serveraddr, serverport)
        self._db=self._client[database]
        self._testname=testname 
        self._sensors=self._db['sensor'+self._testname]
        self._formula_id = (None,)
        self._state = FormulaState(None, None, None, self._formula_id)
        self._handler = RAPLFormulaHWPCReportHandler(None)
        
    def gettimetamps(self,containername): 
        containerdata=list(self._sensors.find({'target':containername},projection=['timestamp']))
        begintime= containerdata[0]['timestamp']
        endtime=containerdata[-1]['timestamp']
        return begintime , endtime 

    def _process_power(self,row,socket,event):
        hwpc_report = HWPCReport.deserialize(row)
        for i in self._handler._process_report(hwpc_report, self._state) : 
            if i.metadata['socket']== socket and i.metadata['event']==event : 
                return i.power
        return -1 
    
    def _get_headers(self,row):
        hwpc_report = HWPCReport.deserialize(row)
        x=self._handler._process_report(hwpc_report,self._state)
        return [(i.metadata['socket'],i.metadata['event']) for i in x ]
    
    
    def getpowers(self,containername): 
        #get the power consumption of the whole machine  while the exection  of a container 
        begin , end = self.gettimetamps(containername) 
        x=list(self._sensors.find({'target':'all','timestamp' :{'$gte':begin,'$lte':end}}))
        conso= pd.DataFrame(x)

        headers=self._get_headers(x[0])
        for i in headers: 
            socket,event=i 
            title="{}_{}".format(event.split('_')[-1],socket)
            conso[title]=conso.T.apply(lambda row: self._process_power(row,socket,event))

        return conso.drop(["_id","groups","sensor","target"],axis=1)
    
    def getpowersfromInterval(self,begin,end ): 
        #get the power consumption of the whole machine  between begin and end 
        x=list(self._sensors.find({'target':'all','timestamp' :{'$gte':begin,'$lte':end}}))
        conso= pd.DataFrame(x)

        headers=self._get_headers(x[0])
        for i in headers: 
            socket,event=i 
            title="{}_{}".format(event.split('_')[-1],socket)
            conso[title]=conso.T.apply(lambda row: self._process_power(row,socket,event))

        return conso.drop(["_id","groups","sensor","target"],axis=1)

    def getenergy(self,containername):
        powers =self.getpowers(containername)
        return powers.sum()
   
    def getrecap(self): 
        meausres = self._db['recap'+self._testname].find(projection={'_id': False,'id':False})
        return pd.DataFrame(list(meausres))

# simple example

In [23]:
#getting an instance of our machine 
machine = testCase("tt2080319")


In [24]:
#to print the powers of a docker container named tommti_python3_intArithmetic_50_0
machine.getpowers('tommti_python3_intArithmetic_50_0')


,timestamp,DRAM_0,PKG_0,DRAM_1,PKG_1
0,2019-03-08 13:37:17.345,6.583572,29.674011,6.349091,14.447571
1,2019-03-08 13:37:18.345,6.575226,29.658569,6.336929,14.400757
2,2019-03-08 13:37:19.346,6.599869,29.647522,6.309570,14.386414
3,2019-03-08 13:37:20.346,6.613754,30.260437,6.380035,20.441345
4,2019-03-08 13:37:21.347,6.583984,29.666931,6.390656,15.033997
5,2019-03-08 13:37:22.347,6.603287,29.661865,6.310287,14.375549
6,2019-03-08 13:37:23.348,6.544464,29.661865,6.326187,14.394409
7,2019-03-08 13:37:24.348,6.573334,30.269043,6.392105,15.607910
8,2019-03-08 13:37:25.348,6.583008,30.379700,6.363968,21.900574
9,2019-03-08 13:37:26.349,6.595474,29.917480,6.333908,15.667175


In [25]:
#to print the energy of a docker container named  tommti_python3_intArithmetic_50_0
machine.getenergy('tommti_python3_intArithmetic_50_0')

DRAM_0     328.559555
PKG_0     1494.400330
DRAM_1     317.453461
PKG_1      861.354065
dtype: float64